In [30]:
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize

import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to /home/ahaque2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ahaque2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# news = pd.read_csv('final_dataset/news_text_with_senti_scores.csv')

# Download and place dataset in the dataset/ folder
news_tweets = pd.read_csv('results/news_tweets.csv')
user_resp = pd.read_csv('results/user_response.csv')

In [5]:
news_tweets.columns, news_tweets.shape

(Index(['Unnamed: 0.6', 'Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0',
        'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0.1.1',
        'Unnamed: 0.1.1.1', 'tweet_id', 'conversation_id', 'author_id',
        'created_at', 'geo', 'lang', 'like_count', 'quote_count', 'reply_count',
        'retweet_count', 'source', 'text', 'display_name', 'topics',
        'topic_labels', 'theme', 'pos_senti', 'neu_senti', 'neg_senti',
        'publication', 'date', 'week', 'polIncl', 'hashtag', 'clean_text',
        'proc_text', 'Segmented#', 'processed_text', 'topic_ids', 'subtopic',
        'trump_pos_new', 'trump_neg_new', 'trump_neu_new', 'biden_pos_new',
        'biden_neg_new', 'biden_neu_new', 'Trump_flag', 'Biden_flag',
        'Trump_Biden_flag', 'topic_id'],
       dtype='object'),
 (24584, 49))

In [728]:
news_tweets = news_tweets.drop_duplicates(subset = ['text'])

In [730]:
import re
news_tweets['hashtag'] = news_tweets.text.apply(lambda x: re.findall(r'#(\w+)', x))

In [731]:
import preprocessor as p
#forming a separate feature for cleaned tweets
clean_tweet = []
for i, v in enumerate(news_tweets['text']):
    # print(v)
    clean_tweet.append(p.clean(v))
    
news_tweets['clean_text'] = clean_tweet

In [732]:
def preprocess_data(data):
    
    #Removes Numbers
    data = data.astype(str).str.replace('\d+', '')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer =  TweetTokenizer()

    def lemmatize_text(text):
        return [(lemmatizer.lemmatize(w)) for w \
                       in w_tokenizer.tokenize((text))]
    
    def preprocess(txt):

        stop_words = set(stopwords.words('english'))
        exclude_words = ['donald', 'trump', 'joe', 'biden', 'president']
        return [w for w in txt if (not w.lower() in stop_words and not w.lower() in exclude_words)]
    
    def remove_punctuation(words):
        new_words = []
        # print(words)
        # sys.exit()
        for word in words:
          new_word = re.sub(r'[^\w\s]', '', (word))
          if new_word != '':
             new_words.append(new_word)
        # print(new_words)
        # sys.exit()
        
        return new_words
    
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    words = words.apply(preprocess)
    
    # print(words)
    # sys.exit()
    
    return pd.DataFrame(words)

In [733]:
proc_text = preprocess_data(news_tweets['clean_text'])
# proc_text = preprocess(proc_text.clean_text.to_list())

In [734]:
len(proc_text)

26388

In [735]:
news_tweets['proc_text'] = proc_text

In [736]:
news_tweets = news_tweets.drop_duplicates(subset = ['proc_text'])

In [739]:
len(proc_tweets), news_tweets.shape

(26388, (24584, 47))

In [740]:
proc_tweets = [' '.join(map(str, l)) for l in news_tweets['proc_text']]
news_tweets['processed_text'] = proc_tweets

In [742]:
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

In [743]:
seed_topic_list = [['drug', 'outbreak', 'flu', 'infection', 'contagious', 'treatment', 'prescription', 'covid', 'test', 'virus', 'ventilator', 'deaths', 'cases', 'pandemic', 
                    'epidemic', 'corona', 'coronavirus', 'covid19', 'patients', 'symptom'],
                   ['lockdown', 'shutdown', 'mask', 'distancing', 'masks'],
                   ['vaccine', 'vaccination', 'cure', 'moderna', 'pfizer', 'astrazeneca', 'j&j', 'dose'],
                   ['education', 'school', 'student', 'teacher', 'children', 'homeschool', 'schools', 'students', 'teachers'],
                   ['economy', 'industry', 'business', 'financial', 'finance', 'fiscal', 'economic', 'job', 'jobless', 'investing', 'investor', 'billion', 'gdp', 'debt', 
                    'liquidity', 'inflation', 'stimulus', 'bill', 'stocks', 'market', 'employment', 'unemployment', 'checks', 'cheques', 'recession', 'bull', 'bullish', 'bear',
                    'bearish', 'dow', 's&p', 'nasdaq', 'trade', 'trading', 'tax', 'loan', 'labor', 'buyback', 'selloff', 'wealth', 'wealthy', 'billionare', 'millionare'],
                   ['earth', 'green', 'pollution', 'ozone', 'deforestation', 'greenhouse', 'wildfire', 'climate', 'warming', 'temperature', 'flood', 'drought', 'glacier', 'environment', 'environmental', 'carbon', 'emission', 'gas', 'fracking'],
                   ['capitol', 'riot', 'siege', 'rioter', 'mob'],
                   ['voter', 'absentee', 'ballot', 'fraud', 'mailin', 'stolen', 'voting', 'election', 'black voters'],
                   ['immigration', 'immigrant', 'refugee', 'border', 'wall', 'migration', 'h1b', 'visa'],
                   ['blm', 'floyd', 'police', 'brutality', 'defund', 'protest', 'protesters', 'officer', 'black lives matter', 'injustice', 'racism', 'racial', 'supremacist'],
                   ['abortion', 'wade', 'roe'],
                   ['supreme', 'court', 'coney', 'barret', 'packing', 'justice', 'judge'],
                   ['security', 'military', 'weapons', 'attack', 'defense', 'gun', 'shooting', 'pentagon'],
                   ['international', 'country', 'global', 'china', 'chinese', 'beijing', 'shanghai', 'iran', 'irani', 'iranian', 'tehran', 'afghanistan', 'afghan', 'afghani', 'afghanistani', 
                    'kabul', 'russia', 'russian', 'moscow', 'britain', 'british', 'brit', 'brexit', 'london', 'Korea', 'Korean', 'kim', 'venezuelan', 'venezuela', 'syrian', 'syria'
                    'world', 'worldwide']]

In [770]:
my_stopwords = frozenset(list(["rt","RT", "&", "amp", "&amp", "http","https", "http://", "https://", "fav", "FAV"]))
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words = my_stopwords, min_df=5)

# do the BERT topic modelling
umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

min_clusters = round(len(proc_tweets) * 0.0017)
hdbscan_model = HDBSCAN(min_cluster_size= min_clusters, metric='euclidean', cluster_selection_method='eom', prediction_data=True, min_samples=5)

# sentence_model = SentenceTransformer("all-mpnet-base-v2")
# embeddings = sentence_model.encode(news_list)

#run the model
topic_model = BERTopic(seed_topic_list = seed_topic_list, nr_topics = 'auto', umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, low_memory=True, calculate_probabilities=True)

topics, probs = topic_model.fit_transform(proc_tweets)

In [772]:
news_tweets['topic_id'] = topics

In [773]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,2901,-1_white house_white_house_capitol
1,0,1086,0_voter_voting_fraud_election
2,1,1074,1_opinion_bidens_inauguration_bidens inauguration
3,2,835,2_analysis_coronavirus_response_death
4,3,769,3_black_protest_life_police
...,...,...,...
85,84,51,84_going_im_im going_shut
86,85,49,85_federal_federal judge_administration_judge
87,86,48,86_school_reopen_reopen school_reopening
88,87,48,87_border_wall_border wall_wall street


In [899]:
# news_tweets[news_tweets.topic_id == 26][['text', 'conversation_id']].iloc[0:50]

In [900]:
topic_model.get_topic(46)

[('democratic', 0.09892212918155356),
 ('democratic presidential', 0.08531983082595261),
 ('nominee', 0.07389369425294269),
 ('presidential nominee', 0.07142068448347932),
 ('presidential', 0.05050432466091089),
 ('presumptive democratic', 0.039622404729708495),
 ('presumptive', 0.039158129915054095),
 ('democratic nominee', 0.033198618601624014),
 ('democratic party', 0.025838794167238353),
 ('party', 0.020754158017640245)]

#### Merging similar subtopics and grouping based on similarity after manual inspection

In [910]:
topic_list = {'election': {'fraud': [0, 71], 'electoral': [15], 'pres_deb': [18], 'opinion': [19, 30, 33], 'biden_transition': [1], 'rally': [6], 'general': [45, 52]},
              'covid': {'general': [2], 'mask': [8], 'response': [2, 9], 'vaccine': [14], 'school-reopen': [49, 86], 'treatment': [11], 'positive': [29], 'advisory': [43]},
              'republican': {'general': [5], 'gop': [63], 'william_barr': [35], 'pence': [55]},
              'white_house': {'general': [12], 'staff': [78, 83], 'first_lady': [58]},
              'democrats': {'harris': [13, 76], 'obama': [20], 'general': [37, 46], 'sander': [31], 'palosi': [62]},
              'national': {'security': [23], 'state': [34], 'executive_order': [25], 'admininstration': [32, 64], 'federal': [65, 85]},
              'blm': {'protest': [3]},
              'capitol': {'riot': [17]},
              'supreme_court': {'general': [4, 79]},
              'economy': {'stimulus': [7, 28], 'tax': [36]},
              'international': {'china': [16]},
              'impeachment': {'general': [27]},
              'media':{'twitter': [21], 'live': [24], 'fox': [41], 'usa_today': [77], 'news':[81]},
              'newyork': {'general': [10]},
              'american': {'people': [38, 73, 88], 'america_first': [42]},
              'healthcare': {'general': [51]},
              'conspiracy_theory': {'general':[44],'fact-check':[72, 75], 'perspective': [26]},
              'other': {'general': [22, 39, 40, 47, 48, 50, 53, 54, 56, 57, 59, 60, 61, 66, 67, 68, 69, 70, 74, 80, 84]},
              'immigration': {'general': [82, 87]}
             }

In [22]:
R_news_tweets = news_tweets[((news_tweets['publication'] == 'Breitbart News') | (news_tweets['publication'] == 'Fox News'))]
L_news_tweets = news_tweets[((news_tweets['publication'] == 'CNN') | (news_tweets['publication'] == 'The Washington Post'))]
C_news_tweets = news_tweets[((news_tweets['publication'] == 'Business Insider') | (news_tweets['publication'] ==  'USA Today'))]

In [29]:
# for top in set(news_tweets.topics):
    
#     print("- " * 50)
#     print(top)
#     print(R_news_tweets[R_news_tweets.topics == top][['like_count', 'quote_count', 'reply_count', 'retweet_count']].mean())
#     print(L_news_tweets[L_news_tweets.topics == top][['like_count', 'quote_count', 'reply_count', 'retweet_count']].mean())
#     print("- " * 50)
    

In [24]:
L_news_tweets[news_tweets.topics == 'conspiracy_theory'][['like_count', 'quote_count', 'reply_count', 'retweet_count']].mean()

/tmp/ipykernel_2552712/1728425267.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  L_news_tweets[news_tweets.topics == 'conspiracy_theory'][['like_count', 'quote_count', 'reply_count', 'retweet_count']].mean()


like_count       875.033195
quote_count       54.056017
reply_count      225.809129
retweet_count    293.280083
dtype: float64

In [18]:
pd.set_option('display.max_colwidth', None)
news_tweets[news_tweets.topics == 'conspiracy_theory'][['subtopic', 'text']]


,subtopic,text
194,conspiracy_theory-general,Donald Trump's future on Facebook and Instagram is now up in the air https://t.co/gmKtlWuGcX
220,conspiracy_theory-fact-check,Former President Trump made false claims from the first hours of his term to the last.\n\nHe repeated some of his favorite falsehoods in both his farewell video address and in his final speech at Joint Base Andrews. Here's a fact check. https://t.co/USyI0FJy0q
418,conspiracy_theory-fact-check,"During the debate on the impeachment resolution, some of President Trump's allies were still using many of the same arguments they did a year ago to criticize Democrats and defend Trump's actions.\n\nHere's a fact check of the false and misleading claims. https://t.co/sgmjolSxSK"
818,conspiracy_theory-general,"Analysis: Joe Biden will be president in a month. Until then, the question is how much damage can be done by a vengeful, delusional soon-to-be ex-President swilling conspiracy theories, writes @stcollinson https://t.co/GRWRbqNIbv"
830,conspiracy_theory-perspective,"With his presidency coming to a close, historians and Americans alike are starting to put Trump in historical perspective, CNN's Harry Enten writes\nhttps://t.co/jP6I6bNhCa"
...,...,...
24169,conspiracy_theory-fact-check,RT @carenbohan: Fact Check: Trump claims coronavirus tests are widely available. They're not https://t.co/W1UwskoACX via @mcollinsNEWS @nic…
24514,conspiracy_theory-general,RT @JillDLawrence: By @aslavitt: #Coronavirus isn't about Trump's #StockMarket and 2020 odds. Or at least it shouldn't be. https://t.co/ayJ…
24559,conspiracy_theory-general,"RT @usatodayDC: Trump's family members, such as Donald Trump Jr. and Lara Trump, are tailoring their messages to reach out to an array of v…"
24578,conspiracy_theory-general,"Trump ally Roger Stone is sentenced to 40 months in prison, less than what prosecutors originally recommended before the DOJ backtracked. https://t.co/EVJKIAPJX6"


In [4]:
# Uncomment to save model

# import pickle

# topic_model.save("bertopic/topic_model_news_tweets")
# pickle.dump(topics, open( "bertopic/topics_news_tweets.pickle", "wb" ) )
# # assert topics == new_topics

### Load saved model

In [2]:
from bertopic import BERTopic
import pickle
#Clear cache or restart kernel
my_model = BERTopic.load("../../bertopic/topic_model_news_tweets")

# topic_model.save("bertopic/topic_model_news_tweets")
# pickle.dump(topics, open( "bertopic/topics_news_tweets.pickle", "wb" ) )
# # assert topics == new_topics

In [3]:
my_model.get_topic_info()

,Topic,Count,Name
0,-1,2901,-1_white house_white_house_capitol
1,0,1086,0_voter_voting_fraud_election
2,1,1074,1_opinion_bidens_inauguration_bidens inauguration
3,2,835,2_analysis_coronavirus_response_death
4,3,769,3_black_protest_life_police
...,...,...,...
85,84,51,84_going_im_im going_shut
86,85,49,85_federal_federal judge_administration_judge
87,86,48,86_school_reopen_reopen school_reopening
88,87,48,87_border_wall_border wall_wall street


In [913]:
# topic_list = {'covid': {'general': [16, 22], 'testing': [0], 'analysis': [8], 'mask': [12], 'cases': [15], 'fauci': [45], 'school-reopen': [56]},
#             'election': {'general': [9, 27, 28, 38, 40, 54, 63], 'fraud': [1], 'opinion': [4, 29, 69, 89], 'georgia': [25]},
#             'white_house': {'general': [2], 'staff': [90]},
#             'security': {'general': [3], 'intelligence': [85]},
#             'supreme_court': {'general': [5, 52]},
#             'blm': {'general': [6]},
#             'capitol': {'general': [7], 'impeachment': [68]},
#             'economy': {'stimulus': [10], 'tax': [51], 'stock': [81]},
#             'international': {'china': [14]},
#             'pres_debate': {'general': [17]},
#             'democarts':{'general': [19], 'kamala': [23, 59, 79], 'obama': [84], 'sanders': [32], 'biden': [39, 50, 66], 'pelosi': [75], 'primary': [60]},
#             'republican': {'general': [20], 'pence': [58], 'trump': [67], 'first_lady': [71]},
#             'others': {'general': [11, 13, 21, 24, 26, 30, 31, 33, 35, 36, 37, 43, 44, 46, 47, 48, 49, 53, 57, 61, 64, 70, 72, 73, 76, 77, 78, 82, 87, 88, 91, 92]},
#             'fact-check': {'general': [55], 'misleading': [74]},
#             'healthcare': {'general': [62]},
#             'immigration': {'border': [83]},
#             'social_media': {'general': [34, 86]},
#             'media': {'general': [18, 80, 93]},
#             'america': {'general': [41, 42, 65]}}

In [914]:
# pd.options.display.max_colwidth = 2000
# news_tweets[news_tweets['topic_ids'] == 41].text

In [916]:
# cls_to_fine_top[46]

In [915]:
### topic Ids
topic_cat_dic = dict([(j, i) for i, j in zip([key for key in topic_list], [x for x in range(len(topic_list))])])

### topic to cluster id mapping
topic_cls_map = dict()
for key in topic_list:
    cls_ids = []
    for k in topic_list[key]:
        cls_ids.extend(topic_list[key][k])
    topic_cls_map[key] =  cls_ids
    
cls_to_fine_top = dict()
for top in topic_list:
    for t in topic_list[top]:
        for k in topic_list[top][t]:
            cls_to_fine_top[k] = top + "-" + t
            
tops, fine_top = [], []
for t in topics:
    
    if(t == -1):
        tops.append('none')
        fine_top.append('none')
        continue
    for k in topic_cls_map:
        if(t in topic_cls_map[k]):
            tops.append(k)
    fine_top.append(cls_to_fine_top[t])

In [917]:
# cls_to_fine_top

In [918]:
len(tops), len(topics), len(fine_top)

(24584, 24584, 24584)

In [919]:
news_tweets['topic_ids'] = topics
news_tweets['topics'] = tops
news_tweets['subtopic'] = fine_top

In [920]:
news_tweets.to_csv('dataset/news_tweets.csv')